In [14]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=5d01fae9e85dbf3fc08de1cf83e1a8ad6cb194f69b8ea90846e751ca3e24d63b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [139]:
import pandas as pd
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from langdetect import detect
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [119]:
# load in data
data = pd.read_csv("data.csv")

# print data
data

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,full_content
0,89541,NaN,International Business Times,Paavan MATHEMA,UN Chief Urges World To 'Stop The Madness' Of ...,UN Secretary-General Antonio Guterres urged th...,https://www.ibtimes.com/un-chief-urges-world-s...,https://d.ibtimes.com/en/full/4496078/nepals-g...,2023-10-30 10:12:35.000000,UN Secretary-General Antonio Guterres urged th...,Nepal,UN Secretary-General Antonio Guterres urged th...
1,89542,NaN,Prtimes.jp,NaN,RANDEBOOよりワンランク上の大人っぽさが漂うニットとベストが新登場。,[株式会社Ainer]\nRANDEBOO（ランデブー）では2023年7月18日(火)より公...,https://prtimes.jp/main/html/rd/p/000000147.00...,https://prtimes.jp/i/32220/147/ogp/d32220-147-...,2023-10-06 04:40:02.000000,"RANDEBOO2023718()WEB2023 Autumn Winter \n""Nepa...",Nepal,NaN
2,89543,NaN,VOA News,webdesk@voanews.com (Agence France-Presse),UN Chief Urges World to 'Stop the Madness' of ...,UN Secretary-General Antonio Guterres urged th...,https://www.voanews.com/a/un-chief-urges-world...,https://gdb.voanews.com/01000000-0a00-0242-60f...,2023-10-30 10:53:30.000000,"Kathmandu, Nepal UN Secretary-General Antonio...",Nepal,NaN
3,89545,NaN,The Indian Express,Editorial,Sikkim warning: Hydroelectricity push must be ...,Ecologists caution against the adverse effects...,https://indianexpress.com/article/opinion/edit...,https://images.indianexpress.com/2023/10/edit-...,2023-10-06 01:20:24.000000,At least 14 persons lost their lives and more ...,Nepal,At least 14 persons lost their lives and more ...
4,89547,NaN,The Times of Israel,Jacob Magid,"200 foreigners, dual nationals cut down in Ham...","France lost 35 citizens, Thailand 33, US 31, U...",https://www.timesofisrael.com/200-foreigners-d...,https://static.timesofisrael.com/www/uploads/2...,2023-10-27 01:08:34.000000,"Scores of foreign citizens were killed, taken ...",Nepal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
105370,781108,NaN,The Indian Express,PTI,"Have done no wrong, only did party work, says ...",The High Court today allowed Shivakumar to wit...,https://indianexpress.com/article/cities/banga...,https://images.indianexpress.com/2023/11/DK-Sh...,2023-11-29 10:57:22,Karnataka Deputy Chief Minister D K Shivakumar...,Home,Karnataka Deputy Chief Minister D K Shivakumar...
105371,781129,NaN,Forbes,"Tom Sanderson, Senior Contributor, \n Tom Sand...",FC Barcelona Guarantees $77.6 Million Champion...,FC Barcelona have guaranteed at least $77.6 mi...,https://www.forbes.com/sites/tomsanderson/2023...,https://imageio.forbes.com/specials-images/ima...,2023-11-29 08:41:18,FC Barcelona have guaranteed at least $767.6 m...,Home,FC Barcelona have guaranteed at least $767.6 m...
105372,781235,NaN,NPR,Brigid McCarthy,Three hospitals ignored her gravely ill fiancé...,"Forty years ago, Sarah Lubarsky came home from...",https://www.npr.org/2023/11/29/1215016001/heal...,https://media.npr.org/assets/img/2023/11/23/sa...,2023-11-29 10:01:12,The photo from David and Sarah Lubarsky's wedd...,Home,The photo from David and Sarah Lubarsky's wedd...
105373,781240,NaN,Forbes,"Gary Stern, Contributor, \n Gary Stern, Contri...",Kerber’s Farm: Bringing Farm To Table To Manha...,"A farmstand in Long Island, Kerber’s Farms has...",https://www.forbes.com/sites/garystern/2023/11...,https://imageio.forbes.com/specials-images/ima...,2023-11-29 13:44:33,Kerbers Farm: Bringing Farm To Table To Manhat...,Home,Kerber’s Farm: Bringing Farm To Table To Manha...


In [120]:
# drop all irrelevant data
drop_columns = ['article_id', 'source_id', 'source_name', 'author', 'title', 'url', 'url_to_image', 'content', 'category', 'full_content']

data.drop(columns=drop_columns, inplace=True)

In [121]:
# print updated database
data

,description,published_at
0,UN Secretary-General Antonio Guterres urged th...,2023-10-30 10:12:35.000000
1,[株式会社Ainer]\nRANDEBOO（ランデブー）では2023年7月18日(火)より公...,2023-10-06 04:40:02.000000
2,UN Secretary-General Antonio Guterres urged th...,2023-10-30 10:53:30.000000
3,Ecologists caution against the adverse effects...,2023-10-06 01:20:24.000000
4,"France lost 35 citizens, Thailand 33, US 31, U...",2023-10-27 01:08:34.000000
...,...,...
105370,The High Court today allowed Shivakumar to wit...,2023-11-29 10:57:22
105371,FC Barcelona have guaranteed at least $77.6 mi...,2023-11-29 08:41:18
105372,"Forty years ago, Sarah Lubarsky came home from...",2023-11-29 10:01:12
105373,"A farmstand in Long Island, Kerber’s Farms has...",2023-11-29 13:44:33


In [122]:
# Standardize published_at to date format
data['published_at'] = pd.to_datetime(data['published_at'], errors='coerce').dt.date

# drop nans
data.dropna(inplace = True)

# show new data
data

,description,published_at
0,UN Secretary-General Antonio Guterres urged th...,2023-10-30
1,[株式会社Ainer]\nRANDEBOO（ランデブー）では2023年7月18日(火)より公...,2023-10-06
2,UN Secretary-General Antonio Guterres urged th...,2023-10-30
3,Ecologists caution against the adverse effects...,2023-10-06
4,"France lost 35 citizens, Thailand 33, US 31, U...",2023-10-27
...,...,...
93694,"Today’s photos, a mélange, were sent a few mon...",2023-10-31
93695,A Himalayan lake fed by melting ice just relea...,2023-10-24
93696,A joint survey will be conducted by officials ...,2023-10-24
93697,UN Secretary-General António Guterres began hi...,2023-10-30


In [123]:
# Function to check if a string contains only English characters
def is_english(s):
    return bool(re.match('^[a-zA-Z0-9\s]*$', s))

# Drop rows with non-English descriptions
data = data[data['description'].apply(lambda x: is_english(x))]

# Drop rows with descriptions that say "No description"
data = data[data['description'] != 'No description']

# Drop rows with descriptions that say "Comments"
data = data[data['description'] != 'Comments']

# show new data
data

,description,published_at
27,The Nepal government has accorded all necessar...,2023-10-06
89,Hamas is holding captive 17 Nepali citizens an...,2023-10-07
459,England captain Butler says defending champion...,2023-10-05
461,New Zealand have a stellar record against Indi...,2023-10-21
468,New Zealand had built a strong platform in Dha...,2023-10-22
...,...,...
93245,Facebook parent Meta Platforms will report res...,2023-10-25
93378,Sea trials are scheduled to be held off the Po...,2023-10-05
93405,Lessons from how we tackled acid rain can be a...,2023-10-25
93410,This is a story about a man who battled a train,2023-10-07


In [124]:
# Rename published_at to date
data = data.rename(columns={'published_at': 'Date'})

# Rename published_at to date
data = data.rename(columns={'description': 'News'})

# Convert date to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort by date in ascending order
data = data.sort_values('Date')

# show new data
data

,News,Date
36256,A new museum exhibition in Paris focuses on th...,2023-10-02
43372,Live Aid the musical coming to London decades ...,2023-10-02
45838,UN authorizes Haiti security mission to fight ...,2023-10-02
79994,Proceedings stress importance of mixing rather...,2023-10-02
29023,Breeding beetles belonging to dozens of specie...,2023-10-02
...,...,...
54760,Paragon Fine and Speciality Chemical stock ope...,2023-11-03
54767,Russian finance ministry to increase deferred ...,2023-11-03
54808,Caribbean Development Bank advances climate ch...,2023-11-03
56665,Stick around after the credits,2023-11-03


In [127]:
# move column 'News' to the end
data = data[[col for col in data.columns if col != 'News'] + ['News']]

# reset indexes
data = data.reset_index(drop=True)

data

,Date,News
0,2023-10-02,A new museum exhibition in Paris focuses on th...
1,2023-10-02,Live Aid the musical coming to London decades ...
2,2023-10-02,UN authorizes Haiti security mission to fight ...
3,2023-10-02,Proceedings stress importance of mixing rather...
4,2023-10-02,Breeding beetles belonging to dozens of specie...
...,...,...
905,2023-11-03,Paragon Fine and Speciality Chemical stock ope...
906,2023-11-03,Russian finance ministry to increase deferred ...
907,2023-11-03,Caribbean Development Bank advances climate ch...
908,2023-11-03,Stick around after the credits


In [128]:
# save only first five news headlines for any given date
currDate = None
articleNum = 0

dropRows = []

for i, row in enumerate(data.values):
    if row[0] != currDate:
        currDate = row[0]
        articleNum = 0
    if articleNum >= 5:
        dropRows.append(i)
    articleNum += 1


data = data.drop(dropRows)
data.head(20)

,Date,News
0,2023-10-02,A new museum exhibition in Paris focuses on th...
1,2023-10-02,Live Aid the musical coming to London decades ...
2,2023-10-02,UN authorizes Haiti security mission to fight ...
3,2023-10-02,Proceedings stress importance of mixing rather...
4,2023-10-02,Breeding beetles belonging to dozens of specie...
14,2023-10-03,The Android app update will be available gradu...
15,2023-10-03,The following are five bowlers to watch at the...
16,2023-10-03,How to prepare for the holiday season and addi...
17,2023-10-03,Hedge funds see Mexico and Ecuador as China al...
18,2023-10-03,Diplomatic rapprochement between Mideast rival...


In [129]:
# reset indexes
data = data.reset_index(drop=True)

data

,Date,News
0,2023-10-02,A new museum exhibition in Paris focuses on th...
1,2023-10-02,Live Aid the musical coming to London decades ...
2,2023-10-02,UN authorizes Haiti security mission to fight ...
3,2023-10-02,Proceedings stress importance of mixing rather...
4,2023-10-02,Breeding beetles belonging to dozens of specie...
...,...,...
160,2023-11-03,Shiba Inu netflow spikes as Shibarium enters n...
161,2023-11-03,Bitfinex to delist eight altcoins following co...
162,2023-11-03,Solana has made tremendous recovery this year
163,2023-11-03,5 nations challenge crypto experts and investi...


In [130]:
# download vader lexicon for sentiment analysis
nltk.download('vader_lexicon')

# initialize vader model to determine sentiment
sentiment_analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [131]:
# Define function to get compound sentiment score
def getSentiment(row):
    return sentiment_analyzer.polarity_scores(row['News'])['compound']

# Save and replace each headline with its sentiment
data["News"] = data.apply(getSentiment, axis=1)

# print updated dataframe
display(data)

,Date,News
0,2023-10-02,0.0000
1,2023-10-02,0.0000
2,2023-10-02,-0.0516
3,2023-10-02,0.2500
4,2023-10-02,0.0000
...,...,...
160,2023-11-03,0.0000
161,2023-11-03,0.2500
162,2023-11-03,0.0000
163,2023-11-03,0.0772


In [133]:
# Normalize sentiment scores

# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to your data and transform the 'News' column
data['Normalized_Scores'] = scaler.fit_transform(data[['News']])

# update name of news column to scores
data.rename(columns={'News': 'Scores'}, inplace=True)

# Display updated normalized scores
display(data)

,Date,Scores,Normalized_Scores
0,2023-10-02,0.0000,0.490560
1,2023-10-02,0.0000,0.490560
2,2023-10-02,-0.0516,0.461990
3,2023-10-02,0.2500,0.628980
4,2023-10-02,0.0000,0.490560
...,...,...,...
160,2023-11-03,0.0000,0.490560
161,2023-11-03,0.2500,0.628980
162,2023-11-03,0.0000,0.490560
163,2023-11-03,0.0772,0.533304


In [134]:
# load both to prepare to combine
news = data
stocks = pd.read_csv("S&P 500 Historical Data Test.csv")

# look at data
display(news)
display(stocks)

,Date,Scores,Normalized_Scores
0,2023-10-02,0.0000,0.490560
1,2023-10-02,0.0000,0.490560
2,2023-10-02,-0.0516,0.461990
3,2023-10-02,0.2500,0.628980
4,2023-10-02,0.0000,0.490560
...,...,...,...
160,2023-11-03,0.0000,0.490560
161,2023-11-03,0.2500,0.628980
162,2023-11-03,0.0000,0.490560
163,2023-11-03,0.0772,0.533304


,Date,Price,Open,High,Low,Vol.,Change %
0,11/03/2023,"4,358.30","4,334.20","4,373.60","4,334.20",NaN,0.94%
1,11/02/2023,"4,317.80","4,268.30","4,319.70","4,268.30",NaN,1.89%
2,11/01/2023,"4,237.90","4,201.30","4,245.60","4,197.70",NaN,1.05%
3,10/31/2023,"4,193.80","4,171.30","4,195.60","4,153.10",NaN,0.65%
4,10/30/2023,"4,166.80","4,139.40","4,177.50","4,132.90",NaN,1.20%
5,10/27/2023,"4,117.40","4,152.90","4,156.70","4,103.80",NaN,-0.48%
6,10/26/2023,"4,137.20","4,176.00","4,183.60","4,127.90",NaN,-1.18%
7,10/25/2023,"4,186.80","4,232.40","4,232.40","4,181.40",NaN,-1.43%
8,10/24/2023,"4,247.70","4,235.80","4,259.40","4,219.40",NaN,0.73%
9,10/23/2023,"4,217.00","4,210.40","4,255.80","4,189.20",NaN,-0.17%


In [135]:
# drop volume column from stocks as it is irrelevant and not filled
stocks = stocks.drop('Vol.', axis=1)
display(stocks)

,Date,Price,Open,High,Low,Change %
0,11/03/2023,"4,358.30","4,334.20","4,373.60","4,334.20",0.94%
1,11/02/2023,"4,317.80","4,268.30","4,319.70","4,268.30",1.89%
2,11/01/2023,"4,237.90","4,201.30","4,245.60","4,197.70",1.05%
3,10/31/2023,"4,193.80","4,171.30","4,195.60","4,153.10",0.65%
4,10/30/2023,"4,166.80","4,139.40","4,177.50","4,132.90",1.20%
5,10/27/2023,"4,117.40","4,152.90","4,156.70","4,103.80",-0.48%
6,10/26/2023,"4,137.20","4,176.00","4,183.60","4,127.90",-1.18%
7,10/25/2023,"4,186.80","4,232.40","4,232.40","4,181.40",-1.43%
8,10/24/2023,"4,247.70","4,235.80","4,259.40","4,219.40",0.73%
9,10/23/2023,"4,217.00","4,210.40","4,255.80","4,189.20",-0.17%


In [136]:
# ensure all stock dates are in proper form to align with news data
stocks["Date"] = pd.to_datetime(stocks["Date"])
display(stocks)

,Date,Price,Open,High,Low,Change %
0,2023-11-03,"4,358.30","4,334.20","4,373.60","4,334.20",0.94%
1,2023-11-02,"4,317.80","4,268.30","4,319.70","4,268.30",1.89%
2,2023-11-01,"4,237.90","4,201.30","4,245.60","4,197.70",1.05%
3,2023-10-31,"4,193.80","4,171.30","4,195.60","4,153.10",0.65%
4,2023-10-30,"4,166.80","4,139.40","4,177.50","4,132.90",1.20%
5,2023-10-27,"4,117.40","4,152.90","4,156.70","4,103.80",-0.48%
6,2023-10-26,"4,137.20","4,176.00","4,183.60","4,127.90",-1.18%
7,2023-10-25,"4,186.80","4,232.40","4,232.40","4,181.40",-1.43%
8,2023-10-24,"4,247.70","4,235.80","4,259.40","4,219.40",0.73%
9,2023-10-23,"4,217.00","4,210.40","4,255.80","4,189.20",-0.17%


In [137]:
# get all scores into one row for each date
newNews = pd.DataFrame(columns = ["Date", "Score 1", "Score 2", "Score 3", "Score 4", "Score 5"])
index = -1
currDate = None
scoreNum = 2

for row in news.values:
    if currDate != row[0]:
        scoreNum = 2
        index += 1
        newRow = [pd.to_datetime(row[0]), row[2], 0, 0, 0, 0]
        newNews.loc[index] = newRow
        currDate = row[0]
    else:
        newNews.at[index, f'Score {scoreNum}'] = row[2]
        scoreNum += 1


display(newNews)

,Date,Score 1,Score 2,Score 3,Score 4,Score 5
0,2023-10-02,0.490560,0.490560,0.461990,0.628980,0.490560
1,2023-10-03,0.490560,0.490560,0.855822,0.490560,0.097669
2,2023-10-04,0.744422,0.004485,0.490560,0.490560,0.490560
3,2023-10-05,0.490560,0.490560,0.490560,0.365262,0.302309
4,2023-10-06,0.799181,0.357068,0.490560,0.490560,0.181939
5,2023-10-07,0.490560,0.744588,0.490560,0.326671,0.025359
6,2023-10-08,0.257184,0.799181,0.059631,0.052821,0.207851
7,2023-10-09,0.314379,0.490560,0.137921,0.490560,0.490560
8,2023-10-10,0.678811,0.433697,0.490560,0.490560,0.302309
9,2023-10-11,0.302309,0.490560,0.678811,0.032612,0.490560


In [140]:
# average the 5 scores for a new column
def avg(row):
    return np.average(row[1:])

newNews['Average Score'] = newNews.apply(avg, axis=1)
display(newNews)

,Date,Score 1,Score 2,Score 3,Score 4,Score 5,Average Score
0,2023-10-02,0.490560,0.490560,0.461990,0.628980,0.490560,0.512530
1,2023-10-03,0.490560,0.490560,0.855822,0.490560,0.097669,0.485034
2,2023-10-04,0.744422,0.004485,0.490560,0.490560,0.490560,0.444117
3,2023-10-05,0.490560,0.490560,0.490560,0.365262,0.302309,0.427850
4,2023-10-06,0.799181,0.357068,0.490560,0.490560,0.181939,0.463861
5,2023-10-07,0.490560,0.744588,0.490560,0.326671,0.025359,0.415547
6,2023-10-08,0.257184,0.799181,0.059631,0.052821,0.207851,0.275334
7,2023-10-09,0.314379,0.490560,0.137921,0.490560,0.490560,0.384796
8,2023-10-10,0.678811,0.433697,0.490560,0.490560,0.302309,0.479187
9,2023-10-11,0.302309,0.490560,0.678811,0.032612,0.490560,0.398970


In [141]:
# drop rows for days we do not have stock data
dates = set(stocks['Date'])

dropRows = []

for i, row in enumerate(newNews.values):
    if row[0] not in dates:
        dropRows.append(i)

newNews = newNews.drop(dropRows)
newNews.reset_index(inplace=True, drop=True)

display(newNews)

,Date,Score 1,Score 2,Score 3,Score 4,Score 5,Average Score
0,2023-10-02,0.490560,0.490560,0.461990,0.628980,0.490560,0.512530
1,2023-10-03,0.490560,0.490560,0.855822,0.490560,0.097669,0.485034
2,2023-10-04,0.744422,0.004485,0.490560,0.490560,0.490560,0.444117
3,2023-10-05,0.490560,0.490560,0.490560,0.365262,0.302309,0.427850
4,2023-10-06,0.799181,0.357068,0.490560,0.490560,0.181939,0.463861
5,2023-10-09,0.314379,0.490560,0.137921,0.490560,0.490560,0.384796
6,2023-10-10,0.678811,0.433697,0.490560,0.490560,0.302309,0.479187
7,2023-10-11,0.302309,0.490560,0.678811,0.032612,0.490560,0.398970
8,2023-10-12,0.490560,0.289796,0.190300,0.807209,0.490560,0.453685
9,2023-10-13,0.490560,0.113615,0.490560,0.490560,1.000000,0.517059


In [142]:
# merge both dataframes
merged = stocks.merge(newNews, on='Date')
display(merged)

,Date,Price,Open,High,Low,Change %,Score 1,Score 2,Score 3,Score 4,Score 5,Average Score
0,2023-11-03,"4,358.30","4,334.20","4,373.60","4,334.20",0.94%,0.490560,0.628980,0.490560,0.533304,0.753779,0.579436
1,2023-11-02,"4,317.80","4,268.30","4,319.70","4,268.30",1.89%,0.490560,0.326671,0.628980,0.678811,0.365262,0.498057
2,2023-11-01,"4,237.90","4,201.30","4,245.60","4,197.70",1.05%,0.490560,0.807209,0.547423,0.602569,0.782183,0.645989
3,2023-10-31,"4,193.80","4,171.30","4,195.60","4,153.10",0.65%,0.447816,0.000000,0.490560,0.561431,0.773268,0.454615
4,2023-10-30,"4,166.80","4,139.40","4,177.50","4,132.90",1.20%,0.940479,0.490560,0.060794,0.490560,0.314379,0.459354
5,2023-10-27,"4,117.40","4,152.90","4,156.70","4,103.80",-0.48%,0.490560,0.490560,0.490560,0.690549,0.326671,0.497780
6,2023-10-26,"4,137.20","4,176.00","4,183.60","4,127.90",-1.18%,0.490560,0.339295,0.490560,0.339295,0.490560,0.430054
7,2023-10-25,"4,186.80","4,232.40","4,232.40","4,181.40",-1.43%,0.420630,0.602569,0.490560,0.490560,0.490560,0.498976
8,2023-10-24,"4,247.70","4,235.80","4,259.40","4,219.40",0.73%,0.490560,0.490560,0.405791,0.723936,0.654449,0.553059
9,2023-10-23,"4,217.00","4,210.40","4,255.80","4,189.20",-0.17%,0.490560,0.490560,0.490560,0.561431,0.723936,0.551409


In [143]:
# save merged dataframe as a csv for later
merged.to_csv("testing_set.csv", index=False)